In [1]:
import subprocess
import importlib

# Function to install a package only if not already installed
def install_package(package_name):
    if importlib.util.find_spec(package_name) is None:
        subprocess.check_call(['pip', 'install', package_name], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Install packages if not found
install_package('import-ipynb')
install_package('gspread')
install_package('oauth2client')
install_package('google-api-python-client')
install_package('utm')

In [2]:
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

import import_ipynb
import contextlib
import os
import importlib
import yaml

with contextlib.redirect_stdout(open(os.devnull, "w")), contextlib.redirect_stderr(open(os.devnull, "w")):
    RFQ_Load = importlib.import_module("RFQ_Load")

class Settings:
    """A class to load and access YAML settings with attribute-style access."""
    
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            # Load YAML data and set it as class attributes
            data = yaml.safe_load(file)
            for key, value in data.items():
                setattr(self, key, value)

# Usage example:
def load_settings():
    """Load settings from settings.yaml and return a Settings object."""
    return Settings('settings.yaml')

# Load the settings
settings = load_settings()

# Automation algo

In [3]:
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('client_secret.json', scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=creds)

def read_google_doc(doc_id):
    """Read content from a Google Doc."""
    doc = docs_service.documents().get(documentId=doc_id).execute()
    content = doc.get('body').get('content')
    
    # Extract text from the document content
    text = ''
    for element in content:
        if 'paragraph' in element:
            for paragraph_element in element['paragraph']['elements']:
                if 'textRun' in paragraph_element:
                    text += paragraph_element['textRun']['content']
    return text

def update_google_doc_multiple_replacements(doc_id, replacements):
    """Replace multiple text items in a Google Doc."""
    requests = [
        {
            'replaceAllText': {
                'containsText': {
                    'text': search_text,
                    'matchCase': True
                },
                'replaceText': replacement_text
            }
        }
        for search_text, replacement_text in replacements.items()
    ]
    
    #batch update
    result = docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
    return result

# Weather & Opencage API requests

ref:
> https://opencagedata.com/
> 
> https://openweathermap.org/api
> 
> https://power.larc.nasa.gov/data-access-viewer/

In [4]:
import requests
import utm

# Function to get latitude, longitude, and locality data from OpenCage API
def get_location_data(city, opencage_key):
    geocode_url = "https://api.opencagedata.com/geocode/v1/json"
    params = {
        'q': city,
        'key': opencage_key,
        'no_annotations': 1  # Focus on essential details
    }
    
    response = requests.get(geocode_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]
            return {
                'latitude': location['geometry']['lat'],
                'longitude': location['geometry']['lng'],
                'locality': location['components'].get('city', city)
            }
    print("Failed to retrieve geolocation data.")
    return None

# Function to convert latitude and longitude to UTM coordinates
def convert_to_utm(lat, lon):
    try:
        utm_result = utm.from_latlon(lat, lon)
        
        return {
            'utm_zone': utm_result[2],
            'utm_letter': utm_result[3],
            'northing': utm_result[1],
            'easting': utm_result[0]
        }
        
    except Exception as e:
        print(f"Failed to convert to UTM: {e}")
        return None

# Function to get elevation from Open Elevation API
def get_elevation(lat, lon):
    elevation_url = "https://api.open-elevation.com/api/v1/lookup"
    params = {'locations': f"{lat},{lon}"}
    
    response = requests.get(elevation_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['results'][0]['elevation'] if 'results' in data else None
    print("Failed to retrieve elevation data.")
    return None

def get_humidity(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if response.status_code == 200:
        humidity = data['main']['humidity']
        return humidity
        #print(f"Relative Humidity in {city}: {humidity}%")
    else:
        print("Error retrieving data")

def get_simple_wind_conditions(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Extracting wind conditions
        wind_speed = data['wind']['speed']  # Wind speed in meters per second
        return wind_speed*3.6
    else:
        print("Failed to retrieve wind data.")
        
def get_climate_data(lat, lon):
    # URL for NASA POWER API
    climate_url = f"https://power.larc.nasa.gov/api/temporal/climatology/point"
    params = {
        'parameters': 'T2M',  # T2M represents the 2-meter air temperature
        'community': 'RE',  # RE community is for general use
        'longitude': lon,
        'latitude': lat,
        'format': 'json'
    }
    
    response = requests.get(climate_url, params=params)
    if response.status_code == 200:
        data = response.json()
        
        if 'properties' in data and 'parameter' in data['properties'] and 'T2M' in data['properties']['parameter']:
            temperature_data = data['properties']['parameter']['T2M']
            
            # Calculate mean summer and winter temperatures
            summer_months = ['JUN', 'JUL', 'AUG']
            winter_months = ['DEC', 'JAN', 'FEB']
            
            # Calculate mean summer temperature
            summer_temps = [temperature_data[month] for month in summer_months]
            summer_temp = sum(summer_temps) / len(summer_temps)
            
            # Calculate mean winter temperature
            winter_temps = [temperature_data[month] for month in winter_months]
            winter_temp = sum(winter_temps) / len(winter_temps)
            
            return round(summer_temp, 2), round(winter_temp, 2)
    print("Failed to retrieve climate data.")
    return None, None 

# data class

ref:
> https://www.omnicalculator.com/physics/dew-point

In [5]:
import numpy as np

class data:
    def __init__(self, city, opencage_key, weather_api_key):
        self.city = city
        self.opencage_key = opencage_key
        self.weather_api_key = weather_api_key

        self.location_data = get_location_data(city, opencage_key)
        if self.location_data:
            self.lat = self.location_data['latitude']
            self.lon = self.location_data['longitude']
            
            # Convert to UTM coordinates
            self.utm = convert_to_utm(self.lat, self.lon)

            # Get elevation data
            self.elevation = get_elevation(self.lat, self.lon)

            # Get climate data
            self.summer_temp, self.winter_temp = get_climate_data(self.lat, self.lon)

            # Get humidity data
            self.humidity = get_humidity(city, weather_api_key)

            # Get dew point data
            def get_dew_point(lat, lon, api_key):
                url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
                response = requests.get(url)

                if response.status_code == 200:
                    data = response.json()
                    temperature = data['main']['temp']
                    humidity = self.humidity
                    ###############################
                    a = 17.625
                    b = 243.04 
                    al = np.log(humidity/100) + (a*temperature/(b + temperature))
                    top = b * al
                    bot = a - al
                    dp = top/bot
                    ##############################
                    return dp
                else:
                    return None, None
            self.dew_point = get_dew_point(self.lat, self.lon, weather_api_key)

            # Get wind data
            self.wind = get_simple_wind_conditions(city, weather_api_key)
            
        else:
            self.lat = self.lon = self.utm_data = None
            self.elevation = None
            self.summer_temp = self.winter_temp = None
            self.dew_point = None
            self.humidity = None
            self.wind = None

# Run code to get API pulls

In [6]:
d = data(settings.Locality_input, settings.opencage_key, settings.weather_api_key)

# Logic fill to parse algo

In [7]:
%%capture
import subprocess, importlib
if importlib.util.find_spec('import-ipynb') is None:
    !pip install import-ipynb

import import_ipynb
from RFQ_MEB import *

In [8]:
feed = S[settings.feed_stream]
char = S[settings.char_stream]
csg = S[settings.sg_stream]

In [9]:
replacements = {
    "PROJCODE": settings.PROJCODE,
    "PREPFOR": settings.PREPFOR,
    "PROJNAME": settings.PROJNAME,
    "DATEINPUT": settings.DATEINPUT,
    ########################################
    "Feed_INPUT": str(feed.fluid),
    "Size_INPUT": settings.Size_INPUT,
    ########################################
    "FCz_input": f"{feed.L_dry[FixedCarbon].m:.2f}",
    "VCz_input": f"{feed.L_dry[Volatiles].m:.2f}",
    "Ashz_input": f"{feed.L_dry[Ash].m:.2f}",
    "MCz_input": f"{feed.L[Moisture].m:.2f}",
    ########################################
    "Cx_input": f"{feed.A[Carbon].m:.2f}",
    "H_input": f"{feed.A[Hydrogen].m:.2f}",
    "O_input": f"{feed.A[Oxygen].m:.2f}",
    "N_input": f"{feed.A[Nitrogen].m:.2f}",
    "S_input": f"{feed.A[Sulphur].m:.2f}",
    "Cl_input": f"{feed.A[Chlorine].m:.2f}",
    ########################################
    "BD_input": settings.BD_input,
    "PD_input": settings.PD_input,
    "PSx_input": settings.PSx_input,
    "size80_input": settings.size80_input,
    "flowability_input": settings.flowability_input,
    "repose_input": settings.repose_input,
    "CV_input": f"{feed.CV.m:.2f}",
    ########################################
    "cFCy_input": f"{char.L[FixedCarbon].m:.2f}",
    "cAshy_input": f"{char.L[Ash].m:.2f}",
    "cVCy_input": f"{char.L[Volatiles].m:.2f}",
    "cMCy_input": f"{char.L[Moisture].m:.2f}",
    ########################################
    "cCz_input":f"{char.A[Carbon].m:.2f}",
    "cHz_input": f"{char.A[Hydrogen].m:.2f}",
    "cOz_input": f"{char.A[Oxygen].m:.2f}",
    "cNz_input": f"{char.A[Nitrogen].m:.2f}",
    "cSz_input": f"{char.A[Sulphur].m:.2f}",
    "cSiz_input": f"{char.A[Silicon].m:.2f}",
    "cClz_input": f"{char.A[Chlorine].m:.2f}",
    ########################################
    "cBDz_input": settings.cBDz_input,
    "cPDz_input": settings.cPDz_input,
    "cPSz_input": settings.cPSz_input,
    "c80sizez_input": settings.c80sizez_input,
    "cflowabilityz_input": settings.cflowabilityz_input,
    "creposez_input": settings.creposez_input,
    "cCVz_input": f"{(-char.HHV / char.m).m:.2f}",
    ########################################
    "NOinp": settings.NOinp,
    "DOinp": settings.DOinp,
    "NRinp": settings.NRinp,
    "Miinp": settings.Miinp,
    "Mainp": settings.Mainp,
    ########################################
    "KILNz_input": settings.KILNz_input,
    "Feedrate_INPUT": f"{feed.m.m:.2f}",
    "Charrate_INPUT": f"{char.m.m:.2f}",
    "CSG_INPUT": f"{csg.m.m:.2f}",
    ########################################
    "Locality_input": settings.Locality_input,
    "Elevation_input": str(d.elevation),
    "Lat_input": str(d.lat),
    "Long_input": str(d.lon),
    "UTMgrid_input": (str(d.utm['utm_zone']) + d.utm['utm_letter']),
    "Northing_input": str(d.utm['northing']),
    "Easting_input": str(d.utm['easting']),
    ########################################
    "Area_input": settings.Area_input,
    "TempFacz_input": settings.TempFacz_input,
    "Laydown_input": settings.Laydown_input,
    ########################################
    "MeanTs_inp": str(d.summer_temp),
    "MeanTw_inp": str(d.winter_temp),
    "AnnRF_inp": settings.AnnRF_inp,
    "Hum_inp": str(d.humidity),
    "DDP_inp": f"{d.dew_point:.2f}",
    "WCA_inp": f"{d.wind:.2f}",
    ########################################
}

print(replacements)

{'PROJCODE': 'T3061-1-90-1', 'PREPFOR': 'AVF ENERGY', 'PROJNAME': 'HP2100 XXX TO CHAR & SYNGAS PLANT', 'DATEINPUT': 'XXX 202X', 'Feed_INPUT': 'wood chip', 'Size_INPUT': '50', 'FCz_input': '15.24', 'VCz_input': '83.70', 'Ashz_input': '1.06', 'MCz_input': '31.00', 'Cx_input': '33.38', 'H_input': '10.33', 'O_input': '55.25', 'N_input': '0.21', 'S_input': '0.10', 'Cl_input': '0.00', 'BD_input': '400', 'PD_input': '650', 'PSx_input': 'Coarse Solids', 'size80_input': '50', 'flowability_input': 'Cohesive', 'repose_input': '22.0', 'CV_input': '16.51', 'cFCy_input': '91.31', 'cAshy_input': '6.69', 'cVCy_input': '2.00', 'cMCy_input': '0.00', 'cCz_input': '92.11', 'cHz_input': '0.24', 'cOz_input': '0.96', 'cNz_input': '0.01', 'cSz_input': '0.00', 'cSiz_input': '6.69', 'cClz_input': '0.00', 'cBDz_input': '260', 'cPDz_input': '300', 'cPSz_input': 'Coarse particles', 'c80sizez_input': '20', 'cflowabilityz_input': 'Cohesive', 'creposez_input': '20', 'cCVz_input': '30.37', 'NOinp': '750', 'DOinp': '95

# Run sequence to update

In [10]:
DOCUMENT_ID = settings.doc_id
update_result = update_google_doc_multiple_replacements(DOCUMENT_ID, replacements)